In [296]:
import numpy as np
total = [0]
def my_matmult(m1, m2):
    ans_mat = np.empty((0,m2.shape[1]))
    for row in m1:
        ans_row = np.array([])
        for col in m2.T:
            ans_row = np.append(ans_row, np.dot(row, col))
            total[0] += len(row)*2
        ans_mat = np.vstack((ans_mat,ans_row))
    return ans_mat

a = np.random.rand(3, 2)
b = np.random.rand(2, 3)

print(np.array_equal(a@b, my_matmult(a,b)))


True


In [297]:
def tensor_product(m1, m2):
  ans = np.empty( (0, m1.shape[-2], m2.shape[-1]) )
  for i in range(m1.shape[0]):
    matmult = np.expand_dims(my_matmult(m1[i], m2[i]), axis=0)
    ans = np.vstack((ans, matmult))
  return ans

a = np.random.rand(2, 3, 4)
b = np.random.rand(2, 4, 2)

print(np.allclose(tensor_product(a,b), a @ b))


True


In [298]:
def nd_tensor_product(m1, m2):
  ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
  ans = np.empty(ans_shape)
  for i in range(m1.shape[0]):
    if len(m1.shape) == 3:
        ans[i] = my_matmult(m1[i], m2[i])

    else:
        ans[i] = nd_tensor_product(m1[i], m2[i])
  return ans

a = np.random.rand(12, 4, 64)
b = np.random.rand(12, 64, 4)

total = [0]

print(np.allclose(nd_tensor_product(a,b), a @ b))
print(total)

True
[24576]


# Quantization

In [159]:
import numpy as np
import math

In [318]:
def matix_absmax_quantization(m1, int_type=np.int8):
    data_max = np.iinfo(int_type).max
    norm_range = math.floor((data_max)**(1/2)/ max(m1.shape))
    quantification_constant = norm_range/np.max(abs(m1))
    new_type = (m1 * quantification_constant).astype(int_type)
    assert np.all((new_type >= -1*norm_range) & (new_type <= norm_range))

    return new_type, quantification_constant

In [319]:
def matrix_quant_matmul(m1, m2):
    m1, m1_const = matix_absmax_quantization(m1, qualtization_type)
    m2, m2_const = matix_absmax_quantization(m2, qualtization_type)

    ans = np.matmul(m1,m2)

    ans = (ans.astype(np.float64))/ (m1_const * m2_const)
    return ans

def vector_quant_matmul(m1, m2):
    matrix = np.empty((m1.shape[0], m2.shape[-1]))
    for row_num in range(len(m1)):
        for col_num in range(m2.shape[-1]):
            row = m1[row_num]
            col = m2[:, col_num]
            int_row, row_const = matix_absmax_quantization(row, qualtization_type)
            int_col, col_const = matix_absmax_quantization(col, qualtization_type)
            int_dot = np.dot(int_row, int_col)
            float_dot = int_dot.astype(np.float64)* 1/(row_const * col_const)
            if np.isnan(float_dot):
                float_dot = 0
            matrix[row_num][col_num] = float_dot
    return matrix

def nd_tensor_product(m1, m2, matrix = True):
    ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
    ans = np.empty(ans_shape)
    if len(ans_shape) < 3:
        return matrix_quant_matmul(m1, m2) if matrix else vector_quant_matmul(m1, m2)
    else:
        for i in range(ans_shape[0]):
            ans[i] = nd_tensor_product(m1[i], m2[i])
    return ans


In [324]:
a = np.array([[ 37.82110668, -67.31442379,  69.34831837],
 [-88.51179468,  63.74114025,  4.15519527]])
b = np.array([[ 57.14724296,  78.94985696, -38.34095173, -31.81479472, -69.31694691],
 [-49.47479473 ,-80.60999366,  58.50876067 , 56.40911124,  92.51044183],
 [-94.53736112 ,  8.19402079 ,-82.28626678 , -5.34393604, -47.48347282]])

qualtization_type = np.int64

print(a[0])
print(b[:, 1])
actual = np.dot(a[0], b[:, 1])

m1, m1_const = matix_absmax_quantization(a[0], qualtization_type)
m2, m2_const = matix_absmax_quantization(b[:, 1], qualtization_type)
int_ans = np.dot(m1,m2)

ans = (int_ans.astype(np.float64))/ (m1_const * m2_const)



print(actual)
print(ans)

[ 37.82110668 -67.31442379  69.34831837]
[ 78.94985696 -80.60999366   8.19402079]
8980.427799868674
8980.427781856803


In [330]:
vec_smaller = 0
iterations = 1000
qualtization_type = np.int16

matrix_diff = 0
vec_diff = 0


for _ in range(iterations):
    a = np.random.uniform(-10, 10, size=(2, 10, 4))
    b = np.random.uniform(-10, 10, size=(2, 4, 10))

    actual = a @ b
    print(actual)

    mx_quantized = nd_tensor_product(a,b, matrix = True)
    # print(f"{mx_quantized=}")
    mx_loss = np.abs(np.mean(mx_quantized - actual))
    matrix_diff += mx_loss
    # print(f'matrix loss: {mx_loss}')

    vec_quantized = nd_tensor_product(a,b, matrix = False)
    # print(f"{vec_quantized=}")
    vec_loss = np.abs(np.mean(vec_quantized - actual))
    # print(vec_loss)
    vec_diff += vec_loss
    # print(f'vec loss: {vec_loss}')
    if vec_loss < mx_loss:
        vec_smaller += 1


print(f'% vec better {vec_smaller/iterations}')
print(f'mtx loss {matrix_diff/iterations}')
print(f'vec loss {vec_diff/iterations}')

% vec better 0.0
mtx loss 0.3364990468150585
vec loss 0.3364990468150585


In [322]:
int_dot = np.float64
row_const = 0
col_const = 1
a = int_dot.astype(np.float64)/(row_const * col_const)
print(a)

TypeError: descriptor 'astype' for 'numpy.generic' objects doesn't apply to a 'type' object